<a href="https://colab.research.google.com/github/felipefcunica/cogroo4py/blob/master/Scripts/Analise_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m spacy download pt

     |████████████████████████████████| 21.2 MB 1.2 MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-py3-none-any.whl size=21186281 sha256=4f9a30e7dc9de8e5b8bc3e9cc3bc2e0fb0299663626ba4037f78d243cc83fb2c
  Stored in directory: /tmp/pip-ephem-wheel-cache-0h4rah_i/wheels/c3/f9/0c/5c014a36941a00f5df5fc0756cb961d7c457a978e697a6ce3b
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [1]:
#requeriments.txt

import regex as re
import pickle
import tensorflow as tf
from scipy.sparse import hstack
from spacy.lang.pt.stop_words import STOP_WORDS
import spacy
import string
from keras.preprocessing.sequence import pad_sequences
import os
import numpy as np
#!python -m spacy download pt

class Intros():

###Inicializaçao e configurações###
  def __init__(self,vecCPath,contexPath,vecRPath,retomPath,vecDPath,defesaPath):  
    self.vectorContex,self.contexModel,self.vectorRetom,self.retomadaModel,self.vectorDefesa,self.defesaModel = self.loadModels(vecCPath,contexPath,vecRPath,retomPath,vecDPath,defesaPath)
    self.stop_words,self.pln,self.pontuacoes = self.confgs()
    self.params = {f'vectorContex':self.vectorContex,
                   f'contexModel':self.contexModel,
                   f'vectorRetom':self.vectorRetom,
                   f'retomadaModel':self.retomadaModel,
                   f'vectorDefesa':self.vectorDefesa,
                   f'defesaModel':self.defesaModel,
                   f'stop_words':f'{len(self.stop_words)} palavras adicionadas' ,
                   f'pontuacoes':self.pontuacoes,
                   f'pln':self.pln} 
    return

  def confgs(self): #Estabelece parâmetros de configuação
    self.stop_words = STOP_WORDS
    self.pontuacoes = string.punctuation
    self.pln = spacy.load('pt')
    self.pln.Defaults.stop_words |= {"possuir", "se", "fato", "modo", "e","o" ,"a","r","d",
                                "algum","uma", "haver","outro","s","n", "desssa", "desse",
                                "dessas","desses", "seja","uso", "fato", "será", "muita",
                                "visto", "outro", "maneira", "d", "n", "há", "seja", "muitas",
                                "fica", "si", "passar", "á", "ficar", "causou", "passando", "passou",
                                "perante", "alguém", "terá", "poucos", "tornou", "podendo", "tenham",
                                "ocorre", "voltar", "todavia", "dando", "eram", "tantos", "conseguinte",
                                "necessário", "devido","havia", "feito", "tinha", "faria", "poderiam",
                                "tê", "lo", "ninguém","sendo", "entretanto", "apesar", "melhor", "teriam",
                                "mesma", "colocam", "haver", "s", "leva", "levar"}
    return self.stop_words,self.pln,self.pontuacoes

  def loadModels (self, vecCPath,contexPath,vecRPath,retomPath,vecDPath,defesaPath): #Carrega modelos
    vectorContex = pickle.load(open(vecCPath,'rb'))
    vectorRetom = pickle.load(open(vecRPath,'rb'))
    contexModel = pickle.load(open(contexPath,'rb'))
    retomadaModel = pickle.load(open(retomPath,'rb'))
    vectorDefesa = pickle.load(open(vecDPath,'rb'))
    defesaModel = tf.keras.models.load_model(defesaPath)
    return vectorContex,contexModel,vectorRetom,retomadaModel,vectorDefesa,defesaModel
#####

###Carregamento da redação, texto motivador e tema###
  def loadRMT (self, redacao, motivador, tema): #Carrega componentes de análise
    self.reda = redacao
    self.mot = motivador
    self.tema = tema
    return self.reda, self.mot, self.tema
#####

###Processamento de Dados###
  def preprocess(self,text): #preprocessamento de texto
    result = []
    pos_tag = ['ADJ', 'NOUN', 'VERB', 'PROPN']
    texto = re.sub(u'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ: ]', '', text )
    doc = self.pln(texto.lower())
    for token in doc:
      if(token.text in self.stop_words or token.text in self.pontuacoes):
        continue
      if(token.pos_ in pos_tag):
        result.append(token.text)
      texto = ' '.join([str(elemento) for elemento in result if not elemento.isdigit()]) 
    return texto

  def sepParagr(self): #Separa sessões da redacao
    self.intro = ((self.reda).split('\n'))[0]
    self.conclu = ((self.reda).split('\n'))[-1]
    self.desenv = []
    for i in ((self.reda).split('\n')):
      if i not in self.intro and i not in self.conclu:
        self.desenv.append(i)
    return self.intro,self.desenv,self.conclu

  def sepFrases(self,text): #Separa frases de uma sessão
    frase = [i for i in re.split("[.!?]",text)]
    n_frases = len(frase)
    fraseDic = dict()
    for i in frase:
      fraseDic[i] = len(i.split())
    return frase,n_frases,fraseDic
#####

###Vetorização de Strings###
  def vectString(self,text,vector): #Vetoriza uma string
    if vector == self.vectorContex:
      return self.vectorContex.transform([text])
    elif vector == self.vectorDefesa:
      return pad_sequences(self.vectorDefesa.texts_to_sequences([text]), maxlen=577, dtype='int32', value=0)
    elif vector == self.vectorRetom:
      raise ValueError('Vector de retomada precisa da função hstack(). Utilize Intros.vectorStack(self,txt1,txt2)')
    else:
      raise ValueError('Vector inválido ou não encontrado, verifique com Intros.params e carregue vetores com Intros.loadModels(vecCPath,contexPath,vecRPath,retomPath,vecDPath,defesaPath)') 

  def vectorStack(self,txt1,txt2): #Agrega dois textos em um vetor.
    txt1P,txt2P =[self.preprocess(txt1)],[self.preprocess(txt2)]
    v2agreg = (self.vectorRetom.transform(txt1P),self.vectorRetom.transform(txt2P))
    return hstack(v2agreg)
#####

###Aplicação de modelos preditivos##
  def contexPredict(self,text): #Predição de contexto em uma string.
    vectorT = self.vectString(self.preprocess(text),self.vectorContex)
    contexto = self.contexModel.predict(vectorT)[0]
    proba = f'{round(max(self.contexModel.predict_proba(vectorT)[0])*100,2)}%'
    predi = (contexto,proba)
    return predi

  def retomadaPredict(self,text): #Predição de retomada ao tema em uma string e o tema+texto motivador 
    juntos = f'{self.tema}\n{self.mot}'
    stack = self.vectorStack(text,juntos)
    pred = self.retomadaModel.predict_proba(stack)
    '''if pred[0] == 0: #Não existe retomada
      proba = pred[1][0]
    if pred[0] == 1: #Existe retomada
      proba = pred[1][-1]'''
    if(np.argmax(pred[0]) == 0):
        pred_proba = "%.2f%%" % (pred[0][0]*100)
    elif (np.argmax(pred[0]) == 1):
        pred_proba = "%.2f%%" % (pred[0][1]*100)
    return (np.argmax(pred[0]), pred_proba)

  def defesaPredict(self,text): #Predição de defesa da tese em uma string.
    vector = self.vectString(text,self.vectorDefesa)
    pred = (self.defesaModel.predict(vector,batch_size=1,verbose = 0)[0])
    if(np.argmax(pred) == 0):
        pred_proba = "%.2f%%" % (pred[0]*100)
    elif (np.argmax(pred) == 1):
        pred_proba = "%.2f%%" % (pred[1]*100)
    return (np.argmax(pred),pred_proba)
    
#####

#Funções de Análise.
  def introAnaliser(self): #Analisador de Introduções
    if type(redacao) == str and type(motivador) == str and type(tema) == str: 
      self.sepParagr()
      frases_intro,n_frases_intro,fraseIntro_dic = self.sepFrases(self.intro)
      contexDict,retomDict,defesaDict = {},{},{}
      for frase in frases_intro:
        preFrase = self.preprocess(frase)
        contexDict[frase] = self.contexPredict(preFrase)
        retomDict[frase] = self.retomadaPredict(preFrase)
        defesaDict[frase] = self.defesaPredict(preFrase)
      data = {'intro':self.intro,
              'n_frases_intro':n_frases_intro,
              'n_palavras_intro':fraseIntro_dic,
              'contextos':contexDict,
              'retomada':retomDict,
              'defesa':defesaDict}
    else:
      raise ValueError('Redacao, motivador e/ou tema não inseridos ou inválidos. Tente "Intros.loadRMT(redacao,motivador,tema) utilizando apenas strings."')

    return data
#####

#Funções de utilidade. 
  def __repr__(self): #saída em formato de str
    return f"{self.introAnaliser()}"

  def setParams(self,oldParam,newArg): #altera parâmetros
    if param in list(self.params.keys()):
      self.params[param] = newArg
    else:
      raise ValueError('Parâmetro não encontrado, veja os parâmetros com Intros.params')
#####


In [2]:
redacao = '''A Constituição Federal de 1988, lei suprema no Brasil, vigente ainda nos dias de hoje, prevê em seu artigo 225° o direito ao meio ambiente ecologicamente equilibrado. Conquanto, isso só tem se reverberado nos dias de hoje, pois devido ao quadro de pandemia, a exploração de recursos naturais diminuiu, o que dificulta o pensamento de que isso seja uma nova normalidade. Diante disso faz-se uma análise dos fatores que favorecem esse pensamento. 
Primeiramente, têm-se que a taxa de desemprego, desde que se iniciou o isolamento social, só está aumentando, o que diminui o consumo humano, pois a renda mensal das famílias tende a diminuir e assim, as empresas e indústrias diminuem a sua produção para que não haja nenhum tipo de prejuízo. Desta forma, a matéria-prima necessária também fica menor, diminuindo grandemente os impactos ambientais e é devido a esse fato que a fauna e a flora deixa de ser tão explorada. 
Ademais, seguindo o pensamento de Fabrício Carvalho, as pessoas não compram pois estão sendo forçadas a não comprar, seja pelo desemprego ou pelos altos preços dos produtos, sendo assim, saindo desse cenário de isolamento, a população tende a voltar com um consumo maior, afinal, haverá mais emprego, a renda aumentará e os preços dos produtos decairão.
Contudo, percebe-se que são impactos momentâneos e não se pode afirmar que é uma nova normalidade. Para que essas alterações continuem no cenário pós-pandemia, é necessário uma intervenção do Estado, feita pelo Ministério da Saúde e pelo Ministério do Meio Ambiente, que devem criar campanhas de conscientização para a população, através de palestras ou até mesmo vídeos ao vivo, comprovando através de dados, a melhora que houve para o meio ambiente e para a saúde, afim de convencer cada um a manter os hábitos adquiridos na pandemia. Dessa maneira, a população iria viver com um meio ambiente ecologicamente equilibrado.'''
motivador = '''Pandemia e Meio Ambiente: Impactos momentâneos ou nova 
normalidade?
PESQUISA E INOVAÇÃO
Desde que a Organização Mundial da Saúde (OMS) declarou estado de 
pandemia para a Covid-19, diversas tentativas de conter a disseminação do vírus 
foram propostas e implementadas, como, por exemplo, o isolamento social da 
população. A baixa atividade humana dos últimos meses gerou uma série de 
consequências e impactos, e, no tangente ao meio ambiente, muitas das 
mudanças foram positivas.
O professor do Programa de Pós-graduação em Ecologia, da 
Universidade Federal de Juiz de Fora (UFJF), Fabrício Alvim Carvalho, afirma 
que as medidas de quarentena são positivas não só para a sociedade, mas para 
a fauna e flora silvestres. “É nítida a diminuição da exposição humana à poluição 
ambiental; consequentemente, diminui-se também os problemas respiratórios 
relacionados. De certa maneira, também é positivo para a fauna silvestre, 
igualmente exposta a essas elevadas concentrações de gases.”
O Conselho Central de Controle de Poluição da Índia (CPCB) verificou 
uma mudança significativa na qualidade do ar, que melhorou cerca de 33% entre 
os dias 16 e 27 de março. O país, com aproximadamente 1,4 bilhão de 
habitantes, segue a política de quarentena desde o dia 22 de março. Possíveis 
explicações para a queda da poluição são a considerável redução no tráfego de 
automóveis e a inatividade de indústrias. Uma das maiores consequências 
ocorreu no norte do país, onde moradores conseguem ver o Himalaia – a 200 
quilômetros de distância – pela primeira vez em 30 anos, além de também 
relatarem mais estrelas visíveis.
Queda nos níveis de NO2 e CO2
O fenômeno, no entanto, não é exclusivo da Índia. Imagens de satélite 
mostram que a pandemia do coronavírus está temporariamente diminuindo 
níveis de poluição do ar ao redor do mundo. Especialistas apontam a quarentena 
como o evento de maior escala já registrado em termos de redução de emissões 
industriais. A Agência Espacial Europeia (ESA) detectou ainda uma redução de 
dióxido de nitrogênio (NO2), composto químico que contribui para a poluição 
atmosférica e para a chuva ácida. O NO2 é resultado de emissões de carros e 
outros processos industriais, podendo, entre outras coisas, causar problemas 
respiratórios.
Já em Nova York, pesquisadores apontaram uma queda dos níveis de 
carbono em mais de 50% abaixo da média. Na China, o fechamento de lojas e 
indústrias resultou uma queda de 25% nas emissões de dióxido de carbono 
(CO2), o que equivale a uma redução global de 6%. Na Itália, país que sofre como isolamento social há mais tempo, golfinhos foram filmados nadando no porto 
de Cagliari, capital da ilha de Sardenha. Os canais de Veneza também estão 
consideravelmente mais limpos e cristalinos após uma semana de quarentena, 
estado que não atingia há 60 anos. Pesquisadores explicam que o lodo do rio, 
que geralmente ficava na superfície graças à movimentação de barcos, afundou 
e foi transportado pelo fluxo da água, que também está mais intenso.
Ceticismo
Por outro lado, o professor de Engenharia da UFJF, Cézar Henrique 
Barra, salienta a existência também de impactos ambientais negativos. “O maior 
deve ser na produção de alimentos. As pessoas estão em casa, 
consequentemente consumindo mais comida, água, energia, serviços como 
comunicação, e gerando muitos resíduos. Creio que a geração de lixo seja o 
segundo impacto mais considerável. Em contrapartida, o ar está mais limpo e 
até a água dos rios pode melhorar; um freio necessário para uma sociedade 
imediatista e egocêntrica.”
Essas mudanças ambientais, no entanto, são de teor temporário. São 
alterações de curto prazo, dependentes do que está acontecendo atualmente, 
sem uma garantia de durabilidade, conforme explica Carvalho. “Tenho uma visão 
muito cética e realista quanto aos problemas ambientais da humanidade. Eu não 
acredito, enquanto especialista, que essa pandemia será o suficiente para 
romper esse paradigma de consumo exagerado, que é a principal causa de 
degradação da natureza”, observa.
“Essa quarentena é pura e simplesmente uma imposição forçada por um 
problema de saúde pública. Isso é muito momentâneo e, assim que a pandemia 
acabar, as coisas voltam ao normal” (Fabrício Carvalho)
“É óbvio que uma pequena parte da sociedade, com mais informação e 
um pouco mais de consciência ambiental, vai diminuir seu padrão de consumo. 
Mas, para o geral, é algo momentâneo. Não há o que se comemorar em médio 
ou longo prazo, pois, ao contrário de uma possível paralisação por uma questão 
de conscientização ou uma súbita revolução da consciência ambiental e de 
políticas de sustentabilidade, essa quarentena é pura e simplesmente uma 
imposição forçada por um problema de saúde pública. Isso é muito momentâneo, 
e assim que a pandemia acabar, as coisas voltam ao normal.”
Nova “normalidade”
Por outro lado, Barra acredita que as alterações socioambientais podem 
resistir no período pós-pandemia. “Esses impactos perdurarão por muito tempo 
devido ao novo modelo de sociedade que teremos de aprender a viver. Não 
voltará a ser como antes; a palavra “normalidade” terá que ser relativizada. 
Teremos que nos adaptar a uma nova ordem, a novas formas de 
relacionamentos, sejam profissionais ou pessoais. Mais respeito ao planeta e ao 
próximo.”
Carvalho aponta ainda a maneira como políticas ambientais sempre são 
afetadas em momentos de crise econômica. “Só será possível traçar uma 
estratégia para frear a degradação ambiental se os governos, de maneira geral, 
não afrouxarem suas políticas de meio ambiente. O que temos acompanhado,entretanto, é que sempre que essas crises econômicas aparecem, essas 
políticas são impactadas e afrouxadas. Na última crise, em 2008, os programas 
de redução de CO2 foram fragilizados. Sempre que o mundo é acometido por 
uma crise econômica, há um grande apelo para que a economia seja 
restabelecida de forma muito acelerada, o que gera várias excepcionalidades, 
várias concessões para fragilizar as políticas de meio ambiente.”
“O ser humano tem sido o maior problema do planeta; ele terá essa única
chance de mudar seu status para algo mais humano” (Cezar Barra)
Conforme a pandemia se alastra, medidas de quarentena vêm sendo 
prorrogadas em todo o mundo. Barra traça um possível cenário do que pode 
acontecer caso o isolamento social perdure por muito tempo: “O primeiro 
impacto socioambiental de um afastamento prolongado será na produção de 
alimentos, um fator chave para evitar conflitos sociais. O segundo serão os 
empregos. No caso do ensino, será necessário evoluir a educação das escolas 
públicas com o apoio de canais de televisão e celulares, desde que sejam comuns 
a todas as classes”, ressalta.
“Essas novas formas de relação a distância, sem tanta necessidade de transportes 
e de um crescimento exacerbado, vão favorecer a recuperação dos sistemas 
ambientais. Muitas extinções previstas terão alguma chance de ser revertidas em 
ecossistemas menos alterados e mais equilibrados. Creio que o impacto 
ambiental será positivo e trará alguma perspectiva para as futuras gerações. O ser 
humano tem sido o maior problema do planeta Terra; ele terá essa única chance 
de mudar o seu status para algo mais 
humano.'''
tema = '''Pandemia e Meio Ambiente: Impactos momentâneos ou nova normalidade?'''


In [3]:
modelsRoot = '/content/drive/MyDrive/Spell_AI/Data/modelos/'
fileNames = ['vocabNosso_vectorizer_CountVectorizer',
             'modelo_OneRest_Classifier.txt',
             'vetorizado_counter_vetorizer',
             'retomadaMLP.txt','vetorizado_keras',
             'modeloDefesa']
paramNames = ['vecCPath','contexPath','vecRPath','retomPath','vecDPath','defesaPath']
loader = [(os.path.join(modelsRoot,i)) for i in fileNames]
init_params = dict()
for i in range(len(paramNames)):
  init_params[paramNames[i]]=loader[i]

In [4]:
init_params

{'contexPath': '/content/drive/MyDrive/Spell_AI/Data/modelos/modelo_OneRest_Classifier.txt',
 'defesaPath': '/content/drive/MyDrive/Spell_AI/Data/modelos/modeloDefesa',
 'retomPath': '/content/drive/MyDrive/Spell_AI/Data/modelos/retomadaMLP.txt',
 'vecCPath': '/content/drive/MyDrive/Spell_AI/Data/modelos/vocabNosso_vectorizer_CountVectorizer',
 'vecDPath': '/content/drive/MyDrive/Spell_AI/Data/modelos/vetorizado_keras',
 'vecRPath': '/content/drive/MyDrive/Spell_AI/Data/modelos/vetorizado_counter_vetorizer'}

In [5]:
analisador = Intros(vecCPath=init_params['vecCPath'],
                    contexPath=init_params['contexPath'],
                    vecRPath=init_params['vecRPath'],
                    retomPath =init_params['retomPath'],
                    vecDPath = init_params['vecDPath'],
                    defesaPath = init_params['defesaPath'])

In [6]:
analisador.params

{'contexModel': OneVsRestClassifier(estimator=CalibratedClassifierCV(base_estimator=LinearSVC(random_state=20))),
 'defesaModel': <keras.engine.functional.Functional at 0x7f5ad347b310>,
 'pln': <spacy.lang.pt.Portuguese at 0x7f5ab275d410>,
 'pontuacoes': '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~',
 'retomadaModel': MLPClassifier(hidden_layer_sizes=100),
 'stop_words': '478 palavras adicionadas',
 'vectorContex': CountVectorizer(),
 'vectorDefesa': <keras_preprocessing.text.Tokenizer at 0x7f5accefff90>,
 'vectorRetom': CountVectorizer()}

In [7]:
analisador.loadRMT(redacao,motivador,tema)
analisador.introAnaliser()

{'contextos': {' ': ('descricao', '44.5%'),
  ' Conquanto, isso só tem se reverberado nos dias de hoje, pois devido ao quadro de pandemia, a exploração de recursos naturais diminuiu, o que dificulta o pensamento de que isso seja uma nova normalidade': ('descricao',
   '48.6%'),
  ' Diante disso faz-se uma análise dos fatores que favorecem esse pensamento': ('descricao',
   '46.83%'),
  'A Constituição Federal de 1988, lei suprema no Brasil, vigente ainda nos dias de hoje, prevê em seu artigo 225° o direito ao meio ambiente ecologicamente equilibrado': ('jurismo',
   '60.17%')},
 'defesa': {' ': (0, '71.12%'),
  ' Conquanto, isso só tem se reverberado nos dias de hoje, pois devido ao quadro de pandemia, a exploração de recursos naturais diminuiu, o que dificulta o pensamento de que isso seja uma nova normalidade': (1,
   '90.78%'),
  ' Diante disso faz-se uma análise dos fatores que favorecem esse pensamento': (0,
   '53.05%'),
  'A Constituição Federal de 1988, lei suprema no Brasil, v